In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo

# Load Dataset
online_retail = fetch_ucirepo(id=352)
X1 = online_retail.data.features
y1 = online_retail.data.targets

# Initial Exploration
print(X1.info())
print(X1.describe())

In [ ]:
online_shoppers = fetch_ucirepo(id=468)
X2 = online_shoppers.data.features
y2 = online_shoppers.data.targets

# Initial Exploration
print(X2.info())
print(X2.describe())

In [ ]:
X1 = X1.dropna()
X2 = X2.fillna(0)

In [ ]:
print(X1.columns)

In [ ]:
non_numeric_cols = X1.select_dtypes(exclude=['number']).columns
print(non_numeric_cols)

In [ ]:
encoder = OneHotEncoder()
X1_encoded = encoder.fit_transform(X1[['Description']])  # Replace 'ColumnName' with the actual name

In [ ]:
X1_encoded = encoder.fit_transform(X1[non_numeric_cols])

In [ ]:
from scipy.sparse import hstack

# Assuming X1 is the original DataFrame
non_numeric_cols = X1.select_dtypes(include=['object']).columns  # Identify non-numeric columns
numeric_cols = X1.select_dtypes(include=['number'])              # Select numeric columns

# Assuming X2 is already loaded and preprocessed
non_numeric_cols_X2 = X2.select_dtypes(include=['object']).columns  # Identify non-numeric columns
encoder_X2 = OneHotEncoder()
X2_encoded = encoder_X2.fit_transform(X2[non_numeric_cols_X2])

# One-hot encode non-numeric columns
encoder = OneHotEncoder(sparse_output=True)  # Use sparse output
X1_encoded = encoder.fit_transform(X1[non_numeric_cols])

# Combine numeric data and encoded features
X1_combined_sparse = hstack([numeric_cols, X1_encoded])

# Convert to a dense DataFrame if necessary
X1_final = pd.DataFrame(X1_combined_sparse.toarray(), columns=list(numeric_cols.columns) + list(encoder.get_feature_names_out(non_numeric_cols)))

In [ ]:
def find_optimal_k(data):
    sse = []
    k_range = range(1, 15)
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(data)
        sse.append(kmeans.inertia_)
    
    plt.plot(k_range, sse, marker='o')
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel('SSE')
    plt.title('Elbow Method')
    plt.show()

# Apply Elbow Method
find_optimal_k(X1_encoded)
find_optimal_k(X2_encoded)

In [ ]:
# K-Means with optimal k
optimal_k1 = 5  # Replace with result from Elbow Method
optimal_k2 = 4  # Replace with result from Elbow Method

kmeans1 = KMeans(n_clusters=optimal_k1, random_state=42)
kmeans1.fit(X1_encoded)
labels1 = kmeans1.labels_

kmeans2 = KMeans(n_clusters=optimal_k2, random_state=42)
kmeans2.fit(X2_encoded)
labels2 = kmeans2.labels_

In [ ]:
# Reduce the dataset size for silhouette calculation
from sklearn.utils import shuffle

X1_sample = shuffle(X1_encoded, random_state=42)[:1000]  # Adjust sample size as needed
X2_sample = shuffle(X2_encoded, random_state=42)[:1000]

from sklearn.metrics import silhouette_score

score1 = silhouette_score(X1_sample, labels1[:1000])
score2 = silhouette_score(X2_sample, labels2[:1000])

# Correctly terminate the f-strings
print(f"Silhouette Score for Dataset 1: {score1}")
print(f"Silhouette Score for Dataset 2: {score2}")

In [ ]:
# Apply PCA to reduce dimensions to 2
pca = PCA(n_components=2)
X1_pca = pca.fit_transform(X1_encoded.toarray())  # Convert sparse matrix to dense
X2_pca = pca.fit_transform(X2_encoded.toarray())

# Scatter plot for Dataset 1
plt.scatter(X1_pca[:, 0], X1_pca[:, 1], c=labels1, cmap='viridis', s=50)
plt.title('Clusters for Dataset 1')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.colorbar(label='Cluster Label')
plt.show()

# Scatter plot for Dataset 2
plt.scatter(X2_pca[:, 0], X2_pca[:, 1], c=labels2, cmap='viridis', s=50)
plt.title('Clusters for Dataset 2')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.colorbar(label='Cluster Label')
plt.show()